In [ ]:
from pytz import timezone
import matplotlib.pyplot as plt
import pandas as pd
import time
import datetime as dt
from datetime import datetime
import MetaTrader5 as mt5
import pandas as pd
import pytz
import talib
import numpy

# This will need to be updated as it should come directly from Currency Strength Script
Symbol = input()

#This will need to be updated to take in the date and time from previous script
if not mt5.initialize():
        print("initialize() failed, error code =",mt5.last_error())
        quit()
mt5.login()
timezone = pytz.timezone("UTC")
now = datetime.now(timezone)
start = datetime.now(timezone) - dt.timedelta(days=1)
utc_from = datetime(start.year, start.month, start.day)
utc_to = datetime(now.year, now.month, now.day, now.hour, now.minute, now.second)
rates = mt5.copy_rates_range(Symbol, mt5.TIMEFRAME_M1, utc_from, utc_to)
htf = pd.DataFrame(rates)
htf['time']=pd.to_datetime(htf['time'], unit='s')
start=time.time()
lookback = 15
# create DataFrame out of the obtained data
currency_strength = pd.DataFrame()
rsi_df=pd.DataFrame()

currency_pair =["EURUSD","EURGBP","EURJPY","EURAUD","EURNZD",
"EURCHF","EURCAD","GBPUSD", "GBPJPY","GBPAUD","GBPNZD","GBPCAD",
"GBPCHF","USDCAD", "USDJPY","CHFJPY","CADJPY","NZDJPY","AUDJPY",
"AUDNZD","AUDCAD", "NZDUSD","NZDCAD","NZDCHF","AUDCHF","CADCHF",
"AUDUSD","USDCHF"]

for pair in currency_pair:
    ohlc = mt5.copy_rates_from(pair,mt5.TIMEFRAME_D1, datetime.now(), lookback)
    df = pd.DataFrame(ohlc)
    rsi_df[pair]=talib.RSI(df.close,7)

# Identifying Currency Strength
rsi_df.tail()
strength=pd.DataFrame()
strength["USD"]=1/7*((100-rsi_df.EURUSD)+(100-rsi_df.GBPUSD)+\
    rsi_df.USDCAD+rsi_df.USDJPY+(100-rsi_df.NZDUSD)+\
        (100-rsi_df.AUDUSD)+rsi_df.USDCHF)
strength["EUR"]=1/7*(rsi_df.EURUSD+rsi_df.EURGBP+\
    rsi_df.EURJPY+rsi_df.EURAUD+rsi_df.EURNZD+rsi_df.EURCHF+\
        rsi_df.EURCAD)
strength["GBP"]=1/7*(rsi_df.GBPUSD+rsi_df.GBPJPY+\
    rsi_df.GBPAUD+rsi_df.GBPNZD+rsi_df.GBPCAD+rsi_df.GBPCHF+\
        (100-rsi_df.EURGBP))
strength["CHF"]=1/7*((100-rsi_df.EURCHF)+(100-rsi_df.GBPCHF)+\
    (100-rsi_df.NZDCHF)+(100-rsi_df.AUDCHF)+(100-rsi_df.CADCHF)+\
        rsi_df.CHFJPY+(100-rsi_df.USDCHF))
strength["JPY"]=1/7*((100-rsi_df.EURJPY)+(100-rsi_df.GBPJPY)+\
    (100-rsi_df.USDJPY)+(100-rsi_df.CHFJPY)+(100-rsi_df.CADJPY)+\
        (100-rsi_df.NZDJPY)+(100-rsi_df.AUDJPY))
strength["AUD"]=1/7*((100-rsi_df.EURAUD)+(100-rsi_df.GBPAUD)+\
    (100-rsi_df.AUDJPY)+rsi_df.AUDNZD+rsi_df.AUDCAD+
    rsi_df.AUDCHF+rsi_df.AUDUSD)
strength["CAD"]=1/7*((100-rsi_df.EURCAD)+(100-rsi_df.GBPCAD)+\
    (100-rsi_df.USDCAD)+rsi_df.CADJPY+(100-rsi_df.AUDCAD)+\
        (100-rsi_df.NZDCAD)+rsi_df.CADCHF)
strength["NZD"]=1/7*((100-rsi_df.EURNZD)+(100-rsi_df.GBPNZD)+\
    rsi_df.NZDJPY+rsi_df.NZDUSD+rsi_df.NZDCAD+rsi_df.NZDCHF+\
        (100-rsi_df.AUDNZD))

if strength['EUR'].loc[strength.index[-1]] > strength['USD'].loc[strength.index[-1]]:
    EURUSD = 'Bullish'
else:
    EURUSD = 'Bearish'
if strength['EUR'].loc[strength.index[-1]] > strength['GBP'].loc[strength.index[-1]]:
    EURGBP = 'Bullish'
else:
    EURGBP = 'Bearish'
if strength['EUR'].loc[strength.index[-1]] > strength['JPY'].loc[strength.index[-1]]:
    EURJPY = 'Bullish'
else:
    EURJPY = 'Bearish'
if strength['EUR'].loc[strength.index[-1]] > strength['AUD'].loc[strength.index[-1]]:
    EURAUD = 'Bullish'
else:
    EURAUD = 'Bearish'
if strength['EUR'].loc[strength.index[-1]] > strength['NZD'].loc[strength.index[-1]]:
    EURNZD = 'Bullish'
else:
    EURNZD = 'Bearish'
if strength['EUR'].loc[strength.index[-1]] > strength['CHF'].loc[strength.index[-1]]:
    EURCHF = 'Bullish'
else:
    EURCHF = 'Bearish'
if strength['EUR'].loc[strength.index[-1]] > strength['CAD'].loc[strength.index[-1]]:
    EURCAD = 'Bullish'
else:
    EURCAD = 'Bearish'

if strength['GBP'].loc[strength.index[-1]] > strength['USD'].loc[strength.index[-1]]:
    GBPUSD = 'Bullish'
else:
    GBPUSD = 'Bearish'
if strength['GBP'].loc[strength.index[-1]] > strength['JPY'].loc[strength.index[-1]]:
    GBPJPY = 'Bullish'
else:
    GBPJPY = 'Bearish'
if strength['GBP'].loc[strength.index[-1]] > strength['AUD'].loc[strength.index[-1]]:
    GBPAUD = 'Bullish'
else:
    GBPAUD = 'Bearish'
if strength['GBP'].loc[strength.index[-1]] > strength['NZD'].loc[strength.index[-1]]:
    GBPNZD = 'Bullish'
else:
    GBPNZD = 'Bearish'
if strength['GBP'].loc[strength.index[-1]] > strength['CHF'].loc[strength.index[-1]]:
    GBPCHF = 'Bullish'
else:
    GBPCHF = 'Bearish'
if strength['GBP'].loc[strength.index[-1]] > strength['CAD'].loc[strength.index[-1]]:
    GBPCAD = 'Bullish'
else:
    GBPCAD = 'Bearish'

if strength['USD'].loc[strength.index[-1]] > strength['CAD'].loc[strength.index[-1]]:
    USDCAD = 'Bullish'
else:
    USDCAD = 'Bearish'
if strength['USD'].loc[strength.index[-1]] > strength['JPY'].loc[strength.index[-1]]:
    USDJPY = 'Bullish'
else:
    USDJPY = 'Bearish'
if strength['USD'].loc[strength.index[-1]] > strength['CHF'].loc[strength.index[-1]]:
    USDCHF = 'Bullish'
else:
    USDCHF = 'Bearish'

if strength['CAD'].loc[strength.index[-1]] > strength['JPY'].loc[strength.index[-1]]:
    CADJPY = 'Bullish'
else:
    CADJPY = 'Bearish'
if strength['CHF'].loc[strength.index[-1]] > strength['JPY'].loc[strength.index[-1]]:
    CHFJPY = 'Bullish'
else:
    CHFJPY = 'Bearish'
if strength['NZD'].loc[strength.index[-1]] > strength['JPY'].loc[strength.index[-1]]:
    NZDJPY = 'Bullish'
else:
    NZDJPY = 'Bearish'
if strength['AUD'].loc[strength.index[-1]] > strength['JPY'].loc[strength.index[-1]]:
    AUDJPY = 'Bullish'
else:
    AUDJPY = 'Bearish'
if strength['AUD'].loc[strength.index[-1]] > strength['NZD'].loc[strength.index[-1]]:
    AUDNZD = 'Bullish'
else:
    AUDNZD = 'Bearish'
if strength['AUD'].loc[strength.index[-1]] > strength['CAD'].loc[strength.index[-1]]:
    AUDCAD = 'Bullish'
else:
    AUDCAD = 'Bearish'

if strength['NZD'].loc[strength.index[-1]] > strength['USD'].loc[strength.index[-1]]:
    NZDUSD = 'Bullish'
else:
    NZDUSD = 'Bearish'
if strength['NZD'].loc[strength.index[-1]] > strength['CHF'].loc[strength.index[-1]]:
    NZDCHF = 'Bullish'
else:
    NZDCHF = 'Bearish'
if strength['NZD'].loc[strength.index[-1]] > strength['CAD'].loc[strength.index[-1]]:
    NZDCAD = 'Bullish'
else:
    NZDCAD = 'Bearish'

if strength['AUD'].loc[strength.index[-1]] > strength['USD'].loc[strength.index[-1]]:
    AUDUSD = 'Bullish'
else:
    AUDUSD = 'Bearish'
if strength['AUD'].loc[strength.index[-1]] > strength['CHF'].loc[strength.index[-1]]:
    AUDCHF = 'Bullish'
else:
    AUDCHF = 'Bearish'
if strength['CAD'].loc[strength.index[-1]] > strength['CHF'].loc[strength.index[-1]]:
    CADCHF = 'Bullish'
else:
    CADCHF = 'Bearish'

MS = eval(Symbol)
rates = mt5.copy_rates_range(Symbol, mt5.TIMEFRAME_M15, utc_from, utc_to)
htf = pd.DataFrame(rates)
htf['time']=pd.to_datetime(htf['time'], unit='s')

# Identifying Market Shift
for i in range(2,htf.shape[0]):
    current = htf.iloc[i,:]
    prev = htf.iloc[i-1,:]
    prev_2 = htf.iloc[i-2,:]
    realbody = abs(current['open'] - current['close'])
    candle_range = current['high'] - current['low']
    idx = htf.index[i]
    htf.loc[idx,'Bullish swing'] = current['low'] > prev['low'] and prev['low'] < prev_2['low']
    htf.loc[idx,'Bearish swing'] = current['high'] < prev['high'] and prev['high'] > prev_2['high']
    #Finding the Bullish Market Shift
    if htf.loc[idx,'Bullish swing'] == True:
        htf.loc[idx,'Bull1'] = htf['low'].loc[htf.index[i-1]]
        Bull1 = htf.loc[htf.index[i-1]]
    if htf.loc[idx,'Bearish swing'] == True and Bull1['time'] < htf.loc[idx,'time']:
        htf.loc[idx,'Bull2'] = htf['high'].loc[htf.index[i-1]]
        Bull2 = htf.loc[htf.index[i-1]]
    if htf.loc[idx,'Bullish swing'] == True and Bull1['time'] < htf.loc[idx,'time']:
        htf.loc[idx,'Bull3'] = htf['low'].loc[htf.index[i-1]]
        Bull3 = htf.loc[htf.index[i-1]]
    #Finding the Bearish Market Shift
    if htf.loc[idx,'Bearish swing'] == True:
        htf.loc[idx,'Bear1'] = htf['high'].loc[htf.index[i-1]]
        Bear1 = htf.loc[htf.index[i-1]]
    if htf.loc[idx,'Bearish swing'] == True and Bear1['time'] < htf.loc[idx,'time']:
        htf.loc[idx,'Bear2'] = htf['low'].loc[htf.index[i-1]]
        Bear2 = htf.loc[htf.index[i-1]]
    if htf.loc[idx,'Bearish swing'] == True and Bear1['time'] < htf.loc[idx,'time']:
        htf.loc[idx,'Bear3'] = htf['high'].loc[htf.index[i-1]]
        Bear3 = htf.loc[htf.index[i-1]]

if MS == 'Bullish':
    if htf.loc[idx,'Bullish swing'] == True:
        bull_low1 = Bull1['low']
    if Bull1['time'] < Bull2['time']:
        bull_high = Bull2['high']
        bulldis_high = bull_high
    if Bull2['time'] < Bull3['time'] == True:
        bull_low2 = Bull3['low']
        bulldis_low = bull_low2
    if Bull1['low'] > Bull3['low']:
        start_time = Bull1['time']
        end_time = Bull3['time']
        print ('Start time is:',start_time)
        print ('End time is:',end_time)
        print ('Price Entry Zone is from', bulldis_low, 'to', bulldis_high)
    else:
        print ("No Pattern Found!")
else:
    if htf.loc[idx,'Bearish swing'] == True:
        Bear_low1 = Bear1['high']
    if Bull1['time'] < Bear2['time']:
        Bear_high = Bear2['low']
        beardis_high = Bear_high
    if Bear2['time'] < Bear3['time'] == True:
        Bear_low2 = Bear3['high']
        beardis_low = Bear_low2
    if Bear1['low'] > Bear3['low']:
        start_time = Bear1['high']
        end_time = Bear3['time']
        print ('Start time is:',start_time)
        print ('End time is:',end_time)
        print ('Price Entry Zone is from', beardis_low, 'to', beardis_high)
    else:
        print ("No Pattern Found!")

# Identifying Fair Value Gap
rates = mt5.copy_rates_range(Symbol, mt5.TIMEFRAME_M1, utc_from, utc_to)
htf = pd.DataFrame(rates)
htf['time']=pd.to_datetime(htf['time'], unit='s')
for i in range(2,htf.shape[0]):
    curr = htf.iloc[i,:]
    pre = htf.iloc[i-1,:]
    pre_2 = htf.iloc[i-2,:]
    pre_3 = htf.iloc[i-3,:]
    realbody = abs(curr['open'] - curr['close'])
    candle_range = curr['high'] - curr['low']
    idx = htf.index[i]
    htf.loc[idx,'Bullish FVG'] = curr['low'] > pre_2['high'] and curr['high'] > pre['high'] and pre['high'] > pre_2['high']
    htf.loc[idx,'Bearish FVG'] = curr['high'] < pre_2['low'] and curr['low'] < pre['low'] and pre['low'] < pre_2['low']
    if curr['Bullish FVG'] == True:
        htf.loc[idx,'PEP'] = htf.loc[idx, 'low']
        Bull_PEP= htf.loc[idx, 'low']
    elif curr['Bearish FVG'] == True:
        htf.loc[idx,'PEP'] = htf.loc[idx, 'high']
        Bear_PEP= htf.loc[idx, 'high']

if MS == 'Bullish':
    print(Bull_PEP)
elif MS == 'Bearish':
    print(Bear_PEP)